<a href="https://colab.research.google.com/github/toni-santos/FEUP-MEST-Proj1/blob/main/task3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install ultralytics
!pip install pascal-voc

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 779.6/779.6 kB 9.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.20.5-py3-none-manylinux2014_x86_64.whl (176.2 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-m

We are expecting for the dataset to be already uploaded to the account's Google Drive, saving time in downloading it over and over again.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
ZIP_PATH = "/content/drive/MyDrive/lego-photos.zip"

import zipfile
with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall("/content/data")

Now we need to "fix" the dataset, separating labels and organizing it into train/test/validation splits

In [ ]:
import os
import shutil
import pandas as pd
from pascal import annotation_from_xml

SPLIT_FILE = '/content/drive/MyDrive/train_test_split.csv'
SRC_DIR = '/content/data/'
DEST_DIR = '/content/dataset/'
label_map = {"lego": 0, "legod": 0}

# Read the CSV file
df = pd.read_csv(SPLIT_FILE)

# Ensure the destination directories exist
test_dir_images = os.path.join(DEST_DIR, 'test/images')
test_dir_labels = os.path.join(DEST_DIR, 'test/labels')
train_dir_images = os.path.join(DEST_DIR, 'train/images')
train_dir_labels = os.path.join(DEST_DIR, 'train/labels')
os.makedirs(test_dir_images, exist_ok=True)
os.makedirs(test_dir_labels, exist_ok=True)
os.makedirs(train_dir_images, exist_ok=True)
os.makedirs(train_dir_labels, exist_ok=True)

# Iterate over each row in the DataFrame
for index, row in df.iterrows():
    filename = row['filename']
    is_test = row['is_test']

    # Determine the source and destination paths
    img_filename = filename.split("/")[-1]
    label_filename = ''.join(img_filename.split(".")[:-1]) + ".xml"

    filepath = '/'.join(filename.split("/")[:-1]) + '/'

    # SRC
    img_path = os.path.join(SRC_DIR, filepath, img_filename)
    label_path = os.path.join(SRC_DIR, filepath, label_filename)

    # DEST
    if is_test == 1:
        img_dest_path = os.path.join(test_dir_images)
        label_dest_path = os.path.join(test_dir_labels)
    else:
        img_dest_path = os.path.join(train_dir_images)
        label_dest_path = os.path.join(train_dir_labels)


    # Copy the file
    if os.path.exists(img_path):
        shutil.copy(img_path, img_dest_path)

        ann = annotation_from_xml(label_path)
        yolo_ann = ann.to_yolo(label_map)
        with open(label_dest_path + "/" + ''.join(img_filename.split(".")[:-1]) + ".txt", "w") as f:
            f.write(yolo_ann)

        print(f"Copied {img_path} to {img_dest_path}")
    else:
        print(f"⚠️ File {img_path} does not exist")


Copied /content/data/photos/1/0_022X_original_1608916900045.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_08Kx_original_1609011668789.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_0GhB_original_1608917422072.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_0MWp_original_1608917351343.jpg to /content/dataset/test/images
Copied /content/data/photos/1/0_0TQm_original_1609011349813.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_0XeN_original_1608915386059.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_1JNv_original_1608915001732.jpg to /content/dataset/test/images
Copied /content/data/photos/1/0_1LiU_original_1609011381473.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_1msQ_original_1608998440922.jpg to /content/dataset/train/images
Copied /content/data/photos/1/0_1OOm_original_60599_1609948720104.jpg to /content/dataset/train/images
Copied /content/data/photo

In [ ]:
import yaml

SRC_DIR = '/content/dataset/'
FILENAME = 'dataset.yaml'
content = {
    'path': '../dataset',
    'train': './train',
    'val': './test',
    'names': {
        0: 'lego'
    }
}

file_path = os.path.join(SRC_DIR, FILENAME)

with open(file_path, 'w') as yaml_file:
    yaml.dump(content, yaml_file, default_flow_style=False)

print(f"YAML file created at {file_path}")

YAML file created at /content/dataset/dataset.yaml


# Detection

In [5]:
from ultralytics import YOLO

DATA_PATH = "/content/dataset/dataset.yaml"
EPOCHS = 15 # TODO: Change this to 10 or 15

In [6]:
detection_model = YOLO("yolov8m.pt") # TODO: Change this to s or m
detection_model.to('cuda')

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [7]:
detection_model.train(data=DATA_PATH, epochs=EPOCHS, plots=True, single_cls=True, save=True)

engine/trainer: task=detect, mode=train, model=yolov8m.pt, data=/content/dataset/dataset.yaml, epochs=15, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda:0, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=True, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


train: Scanning /content/dataset/train/labels.cache... 2659 images, 0 backgrounds, 0 corrupt: 100%|██████████| 2659/2659 [00:00<?, ?it/s]


albumentations: Blur(p=0.01, blur_limit=(3, 7)), MedianBlur(p=0.01, blur_limit=(3, 7)), ToGray(p=0.01), CLAHE(p=0.01, clip_limit=(1, 4.0), tile_grid_size=(8, 8))


...

                   all        274        614      0.957      0.945      0.973      0.897



15 epochs completed in 3.029 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 52.0MB
Optimizer stripped from runs/detect/train/weights/best.pt, 52.0MB

Validating runs/detect/train/weights/best.pt...
Ultralytics YOLOv8.2.28 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 218 layers, 25840339 parameters, 0 gradients, 78.7 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 9/9 [00:52<00:00,  5.88s/it]


                   all        274        614      0.957      0.945      0.973      0.897
Speed: 0.6ms preprocess, 9.6ms inference, 0.0ms loss, 4.8ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7ccbabe49540>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [ ]:
detection_model.val()

# Segmentation

In [11]:
from ultralytics import YOLO
import os
import gc
import numpy as np
import cv2

TEST_DATA = "/content/dataset/test/images"
DEST_WEIGHTS = "/content/segmentation_results/weights"
DEST_GC = "/content/segmentation_results/gc"

os.makedirs(DEST_WEIGHTS, exist_ok=True)
os.makedirs(DEST_GC, exist_ok=True)

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 48, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(48, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(48, 96, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(96, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(192, 96, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(96, eps=0.001, momentum=0.03, affine=True, track_running_

In [ ]:
new_model = YOLO("/content/runs/detect/train/weights/best.pt")
results = new_model.predict(source = TEST_DATA, stream = True)

In [19]:
for result in results:
  if result.boxes.numpy().size() == 0:
    del result
    gc.collect()

  mask_all = np.zeros(img.shape[:2], np.uint8)


image 1/274 /content/dataset/test/images/0_0MWp_original_1608917351343.jpg: 640x480 1 lego, 29.5ms
ultralytics.engine.results.Results object with attributes:

boxes: ultralytics.engine.results.Boxes object
keypoints: None
masks: None
names: {0: 'lego'}
obb: None
orig_img: array([[[ 8, 11, 26],
        [ 8, 11, 26],
        [ 8, 11, 26],
        ...,
        [12, 25, 41],
        [14, 27, 43],
        [15, 28, 44]],

       [[ 8, 11, 26],
        [ 8, 11, 26],
        [ 8, 11, 26],
        ...,
        [11, 24, 40],
        [12, 25, 41],
        [12, 25, 41]],

       [[ 8, 11, 26],
        [ 8, 11, 26],
        [ 8, 11, 26],
        ...,
        [11, 24, 40],
        [10, 23, 39],
        [10, 23, 39]],

       ...,

       [[22, 23, 27],
        [22, 23, 27],
        [22, 23, 27],
        ...,
        [12, 10,  9],
        [12, 10,  9],
        [13, 11, 10]],

       [[22, 23, 27],
        [22, 23, 27],
        [22, 23, 27],
        ...,
        [12, 10,  9],
        [13, 11, 10],
  

NameError: name 'akjsdfadjskn' is not defined

In [15]:
# TODO: auto annotate, train and run

# segmentation_model = YOLO("yolov8s.pt").load("/content/runs/detect/train/weights/best.pt")
# segmentation_model.to('cuda')

# results = segmentation_model.predict(data = TEST_DATA, stream = True)

# for result in results:
#   if result.masks is None:
#     del result
#     gc.collect()

#   img = np.copy(result.orig_img)
#   img_name = Path(result.path).stem
#   mask_all = np.zeros(img.shape[:2], np.uint8)

#   for c in result:
#       # Your contour extraction remains the same
#       contour = c.masks.xy.pop().astype(np.int32).reshape(-1, 1, 2)
#       # Draw each contour onto the common mask
#       cv2.drawContours(mask_all, [contour], -1, (255, 255, 255), cv2.FILLED)

#   mask3ch = cv2.cvtColor(mask_all, cv2.COLOR_GRAY2BGR)
#   isolated = cv2.bitwise_and(mask3ch, img)

#   cv2.imwrite(f"{DEST_WEIGHTS}/{img_name}.jpg", isolated)

#   del img
#   del img_name
#   del mask_all
#   del result
#   del isolated
#   del mask3ch
#   gc.collect()

WARNING ⚠️ 'source' is missing. Using 'source=/usr/local/lib/python3.10/dist-packages/ultralytics/assets'.

image 1/2 /usr/local/lib/python3.10/dist-packages/ultralytics/assets/bus.jpg: 640x480 (no detections), 28.1ms


NameError: name 'result' is not defined

In [ ]:
# TRAIN_SRC_PATH = "/content/dataset/train/images"
# TEST_SRC_PATH = "/content/dataset/test/images"
# TRAIN_DEST_PATH = "/content/segmentation_dataset/train/images"
# TEST_DEST_PATH = "/content/segmentation_dataset/test/images"

# shutil.copytree(TRAIN_SRC_PATH, TRAIN_DEST_PATH)
# shutil.copytree(TEST_SRC_PATH, TEST_DEST_PATH)

'/content/segmentation_dataset/test/images'

In [ ]:
# from ultralytics.data.annotator import auto_annotate

# auto_annotate(data=TRAIN_DEST_PATH, det_model="/content/detection_model.pt" , sam_model='sam_b.pt', device="cuda", output_dir="/content/segmentation_dataset/train/labels")
# auto_annotate(data=TEST_DEST_PATH, det_model="/content/detection_model.pt" , sam_model='sam_b.pt', device="cuda", output_dir="/content/segmentation_dataset/test/labels")

In [ ]:
# segmentation_model = YOLO( PATH TO LAST WEIGHTS )